In [82]:
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor,AdaBoostRegressor, GradientBoostingRegressor

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.tree import DecisionTreeRegressor, export_graphviz
import graphviz

In [30]:
df = pd.read_csv('../data/clean/clean_movies.csv')

In [32]:
df

,Unnamed: 0,title,vote_average,release_date,revenue,runtime,adult,budget,genres
0,3,Avatar,7.573,2009-12-15,2923706026,162,False,237000000,"Action, Adventure, Fantasy, Science Fiction"
1,15,Avengers: Endgame,8.263,2019-04-24,2800000000,181,False,356000000,"Adventure, Science Fiction, Action"
2,282,Avatar: The Way of Water,7.654,2022-12-14,2320250281,192,False,460000000,"Science Fiction, Adventure, Action"
3,56,Star Wars: The Force Awakens,7.293,2015-12-15,2068223624,136,False,245000000,"Adventure, Action, Science Fiction"
4,6,Avengers: Infinity War,8.255,2018-04-25,2052415039,149,False,300000000,"Adventure, Action, Science Fiction"
...,...,...,...,...,...,...,...,...,...
5892,308571,Shrek's Yule Log,10.000,2010-12-07,1000,30,False,1000,"Family, Animation, Comedy"
5893,894592,Grandpa Comes Back from Germany,0.000,2013-08-09,1000,105,False,500,"Crime, Thriller, Action"
5894,449098,Across the Rails,0.000,2022-01-12,1000,40,False,3000,"Documentary, Adventure"
5895,202355,Plane Tree,5.000,2018-10-05,1000,99,False,200000,"Family, Drama, Mystery"


In [60]:
filtered_movies= df.drop(columns=['Unnamed: 0','title','vote_average','release_date','runtime','adult','budget'])

In [62]:
filtered_movies

,revenue,genres
0,2923706026,"Action, Adventure, Fantasy, Science Fiction"
1,2800000000,"Adventure, Science Fiction, Action"
2,2320250281,"Science Fiction, Adventure, Action"
3,2068223624,"Adventure, Action, Science Fiction"
4,2052415039,"Adventure, Action, Science Fiction"
...,...,...
5892,1000,"Family, Animation, Comedy"
5893,1000,"Crime, Thriller, Action"
5894,1000,"Documentary, Adventure"
5895,1000,"Family, Drama, Mystery"


In [64]:
# Split genres column into separate genres and normalize
genres_split = filtered_movies['genres'].str.get_dummies(sep=',')
genres_split.columns = genres_split.columns.str.strip().str.lower()  # Strip spaces and lowercase

filtered_movies_with_genres = pd.concat([filtered_movies, genres_split], axis=1)

# Drop genres column
filtered_movies_with_genres.drop(columns=['genres'], inplace=True)

# Duplicated genre columns sum
genres_split_combined = genres_split.T.groupby(genres_split.columns).sum().T

# Merging Columns with the same genre
filtered_movies_with_genres = pd.concat([filtered_movies.drop(columns='genres'), genres_split_combined], axis=1)

# Count of each genre
genre_counts = genres_split_combined.sum().sort_values(ascending=False)

In [66]:
print("Genre counts:")
print(genre_counts)

filtered_movies_with_genres.head()

Genre counts:
drama              2840
comedy             2009
action             1574
thriller           1407
romance             966
adventure           962
crime               772
horror              658
family              627
fantasy             564
science fiction     521
mystery             465
animation           379
history             326
music               193
war                 191
documentary         161
western              47
tv movie             27
dtype: int64


,revenue,action,adventure,animation,comedy,crime,documentary,drama,family,fantasy,history,horror,music,mystery,romance,science fiction,thriller,tv movie,war,western
0,2923706026,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1,2800000000,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2320250281,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,2068223624,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
4,2052415039,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [70]:
# Features and target
X = filtered_movies_with_genres[["action", "adventure", "animation", "comedy", 
               "crime", "documentary", "drama", "family", 
               "fantasy", "history", "horror", "music", 
               "mystery", "romance", "science fiction", 
               "thriller", "tv movie", "war", "western"]]
# Define the target
filtered_movies_with_genres['best_seller'] = (filtered_movies_with_genres['revenue'] > 1e8).astype(int)
y = filtered_movies_with_genres['best_seller']

#TTs
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [72]:
# Initialize and train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [74]:
# Predictions
y_pred = model.predict(X_test)

# Classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Feature importance
feature_importances = pd.Series(model.feature_importances_, index=X.columns)
print("Feature Importance:")
print(feature_importances.sort_values(ascending=False))

# Predicting the next bestseller genre
predicted_genres = X.columns[(model.feature_importances_ > 0.05)].tolist()
print(f"Predicted next best-selling genres: {predicted_genres}")

Classification Report:
              precision    recall  f1-score   support

           0       0.84      0.95      0.89       932
           1       0.62      0.31      0.41       248

    accuracy                           0.82      1180
   macro avg       0.73      0.63      0.65      1180
weighted avg       0.79      0.82      0.79      1180

Feature Importance:
adventure          0.157925
science fiction    0.095804
action             0.086980
animation          0.079545
fantasy            0.076856
drama              0.075487
comedy             0.070060
family             0.066319
thriller           0.052368
mystery            0.043824
horror             0.037821
romance            0.036706
crime              0.031079
war                0.024476
history            0.022641
music              0.014025
western            0.012811
documentary        0.009223
tv movie           0.006052
dtype: float64
Predicted next best-selling genres: ['action', 'adventure', 'animation', 'comedy', 

In [84]:
# Decision tree visualization
tree = DecisionTreeRegressor(max_depth=2, random_state=42)
tree.fit(X_train, y_train)

dot_data = export_graphviz(tree, out_file=None, filled=True, rounded=True, feature_names=X_train.columns)
graph = graphviz.Source(dot_data)
graph.render("decision_tree")  
graph.view() 

'decision_tree.pdf'